# Monument Classification with Transfer Learning and TensorFlow Lite

1. Starting with a pre-trained model (in this case, MobileNetV2)
2. Fine-tuning it on paris6k
3. Converting the fine-tuned model to TensorFlow Lite format

## Setup

### Install required packages

In [ ]:
!python --version
!pip install --upgrade pip
!pip install tensorflow
!pip install albumentations
!pip install pycocotools

### Import necessary libraries

In [ ]:
from google.colab import drive
import os
import json
import tensorflow as tf
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import cv2

assert tf.__version__.startswith('2')

## Mount Google Drive and Set Paths

### Mount Google Drive

In [ ]:
drive.mount('/content/drive')

### Define paths

In [ ]:
base_path = '/content/drive/MyDrive/'
source_path = base_path + 'Datasets/revisitop/rparis6k/data/'
dest_base_path = base_path + 'MyProject/rparis6k/'

train_dataset_path = dest_base_path + 'train/'
validation_dataset_path = dest_base_path + 'validation/'
test_dataset_path = dest_base_path + 'test/'

## Dataset Preparation

### Copy images

In [ ]:
# Function to copy images
def copy_images(file_list, dest_folder):
    with open(file_list, 'r') as f:
        for line in f:
            img_name = line.strip()
            src = os.path.join(source_path, img_name)
            dst = os.path.join(dest_folder, img_name)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy2(src, dst)

# Copy images for each set
if not os.listdir(train_dataset_path) and not os.listdir(validation_dataset_path) and not os.listdir(test_dataset_path):
    copy_images(dest_base_path + 'train.txt', train_dataset_path + 'images/')
    copy_images(dest_base_path + 'val.txt', validation_dataset_path + 'images/')
    copy_images(dest_base_path + 'test.txt', test_dataset_path + 'images/')
    print("Dataset division completed!\n")
else:
    print("One or more directories are not empty. Copy operation aborted.\n")

print(f"Number of images in train set: {len(os.listdir(train_dataset_path + 'images/'))}")
print(f"Number of images in validation set: {len(os.listdir(validation_dataset_path + 'images/'))}")
print(f"Number of images in test set: {len(os.listdir(test_dataset_path + 'images/'))}")

### Review dataset

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

### Data Augmentation

In [ ]:
# Define Albumentations transformations
train_transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.Transpose(),
    A.OneOf([
        A.IAAAdditiveGaussianNoise(),
        A.GaussNoise(),
    ], p=0.2),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.5, rotate_limit=45, p=0.2),
    A.OneOf([
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=0.1),
        A.IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    A.OneOf([
        A.CLAHE(clip_limit=2),
        A.IAASharpen(),
        A.IAAEmboss(),
        A.RandomBrightnessContrast(),
    ], p=0.3),
    A.HueSaturationValue(p=0.3),
])

def augment_image(image, transform=train_transform):
    image = np.array(image)
    augmented = transform(image=image)
    return augmented['image']


## Dataset Creation

In [ ]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image

def preprocess_and_augment(image, label):
    image = tf.numpy_function(augment_image, [image], tf.float32)
    image.set_shape([224, 224, 3])
    return image, label

def get_dataset(image_paths, labels, batch_size, is_training=False):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (load_and_preprocess_image(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    if is_training:
        dataset = dataset.map(preprocess_and_augment, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

## Load the dataset

In [ ]:
image_paths, labels = load_your_dataset()  # TODO: implement this function

In [ ]:
# This function should return two lists: one containing the paths to the images,
# and another containing the corresponding labels (as one-hot encoded vectors).

### Split the dataset

In [ ]:
# Split the dataset
split_index = int(len(image_paths) * 0.8)
image_paths_train = image_paths[:split_index]
labels_train = labels[:split_index]
image_paths_val = image_paths[split_index:]
labels_val = labels[split_index:]

train_dataset = get_dataset(image_paths_train, labels_train, batch_size=32, is_training=True)
val_dataset = get_dataset(image_paths_val, labels_val, batch_size=32, is_training=False)

## Model

### Model creations

In [ ]:
# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(13, activation='softmax')(x)  # 13 classes (12 monuments + background)

model = Model(inputs=base_model.input, outputs=predictions)

### Model compilation

In [1]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'model' is not defined

### Fine-tuning

In [ ]:
# Unfreeze the top layers of the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
model.fit(...)

## Training

- Start by training for a few epochs and monitor the validation loss and accuracy.
- If the model is underfitting (high training and validation loss):
  1. Unfreeze more layers of the base model
  2. Train for more epochs
  3. Increase model capacity (add more dense layers)
- If the model is overfitting (low training loss, high validation loss):
  1. Add regularization (e.g., dropout layers)
  2. Use data augmentation (already implemented)
  3.  Reduce model capacity

- Start with a small number of epochs (e.g., 10) and monitor the training and validation metrics.
- Gradually increase the number of epochs if needed.
- Use the ModelCheckpoint callback to save the best model based on validation accuracy.
- Use the EarlyStopping callback to prevent overfitting by stopping training when the validation loss stops improving.

### Resume training

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('best_model.h5')

# Continue training
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    initial_epoch=history.epoch[-1],  # Start from the last epoch
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy'),
        tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
    ]
)

### Train the model

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy'),
        tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
    ]
)

## Evaluate

### Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation loss: {loss}")
print(f"Validation accuracy: {accuracy}")

## Export to TensorFlow Lite

### Convert to TFLite

In [ ]:
model.save('monumenti_model.h5') # TODO: check line

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

### Save the TFLite model

In [ ]:
with open('monuments_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Quantization

### Quantize the model

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quantized = converter.convert()

### Save the quantized TFLite model

In [ ]:
with open('monuments_model_quantized.tflite', 'wb') as f:
    f.write(tflite_model_quantized)